<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [ ]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [ ]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://github.com/hossiiii/quick_learning_symbol_hoshoku/assets/47712051/92b1753c-f270-4b9e-af17-229cc6dbe094">

<hr />

# 12.検証

ブロックチェーン上に記録されたさまざまな情報を検証します。 ブロックチェーンへのデータ記録は全ノードの合意を持って行われますが、 ブロックチェーンへのデータ参照はノード単体からの情報取得であるため、 信用できないノードの情報を元にして新たな取引を行いたい場合は、ノードから取得したデータに対して検証を行う必要があります。


## 13.1 トランザクションの検証

トランザクションがブロックヘッダーに含まれていることを検証します。この検証が成功すれば、トランザクションがブロックチェーンの合意によって承認されたものとみなすことができます。

本章のサンプルスクリプトを実行する前に以下を実行して必要ライブラリを読み込んでおいてください。


In [ ]:
Buffer = require("/node_modules/buffer").Buffer;
cat = require("/node_modules/catbuffer-typescript");
sha3_256 = require('/node_modules/js-sha3').sha3_256;

accountRepo = repo.createAccountRepository();
blockRepo = repo.createBlockRepository();
stateProofService = new sym.StateProofService(repo);

### 検証するペイロード

今回検証するトランザクションペイロードとそのトランザクションが記録されているとされるブロック高です。

In [ ]:
payload = '6801000000000000702A558251B6319A90FA15640BDCB0A62737567D7C6F32036C566B029EC2B89D9E7ABC676A689DA9D4672927BD26CB2EE02B7875AD08FF630ED62B81224B2F059DC9CD2B71936107BD31D3BE460B0601FE0792F6BF5BF45ED72B77BA00DFB43B0000000002984141A08C000000000000C268FCAD06000000B969193AA7F8091B3E64581078D175A3140E346D3B73284B754063079BF29B84580000000000000058000000000000009DC9CD2B71936107BD31D3BE460B0601FE0792F6BF5BF45ED72B77BA00DFB43B00000000019844419827DC93C158BBE73CA5D179E62359113772BFCF82F5B5D5D72781051BB772970000040000000000000000000000000058852861225DB34ADE78D16CFFC4B092082EAF24DCC9967D2C1D81039FD9C86C069B56E288CF53A395091937F74952399F7BD9B22C703611236CE309170B65E915187E5D5236CB3100143F4F1085C272FB6B82C490BF99ADFDB1130124CB3C0B';
height = 853913;

###### 出力例
```js
> AggregateTransaction
    cosignatures: []
    deadline: Deadline {adjustedValue: 12197090355}
  > innerTransactions: Array(2)
      0: TransferTransaction {type: 16724, networkType: 152, version: 1, deadline: Deadline, maxFee: UInt64, …}
      1: TransferTransaction {type: 16724, networkType: 152, version: 1, deadline: Deadline, maxFee: UInt64, …}
    maxFee: UInt64 {lower: 44800, higher: 0}
    networkType: 152
    payloadSize: undefined
    signature: "4999A8437DA1C339280ED19BE0814965B73D60A1A6AF2F3856F69FBFF9C7123427757247A231EB89BB8844F37AC6F7559F859E2FDE39B8FA58A57F36DDB3B505"
    signer: PublicAccount
      address: Address {address: 'TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ', networkType: 152}
      publicKey: "D4933FC1E4C56F9DF9314E9E0533173E1AB727BDB2A04B59F048124E93BEFBD2"
    transactionInfo: undefined
    type: 16705
    version: 1
```

念のため、Aliceがすでに署名したトランザクション（ペイロード）かどうかを検証します。

In [ ]:
Buffer = require("/node_modules/buffer").Buffer;
res = tx.signer.verifySignature(
    tx.getSigningBytes([...Buffer.from(signedPayload,'hex')],[...Buffer.from(generationHash,'hex')]),
    tx.signature
);
console.log(res);

###### 出力例
```js
> true
```

ペイロードがsigner、つまりAliceによって署名されたものであることが確認できました。
次にBobが連署します。


In [ ]:
bobSignedTx = sym.CosignatureTransaction.signTransactionPayload(bob, signedPayload, generationHash);
bobSignedTxSignature = bobSignedTx.signature;
bobSignedTxSignerPublicKey = bobSignedTx.signerPublicKey;


CosignatureTransactionで署名を行い、bobSignedTxSignature,bobSignedTxSignerPublicKeyを出力しAliceに返却します。  
Bobが全ての署名を揃えられる場合は、Aliceに返却しなくてもBobがアナウンスすることも可能です。

## 12.3 Aliceによるアナウンス

AliceはBobからbobSignedTxSignature,bobSignedTxSignerPublicKeyを受け取ります。  
また事前にAlice自身で作成したsignedPayloadを用意します。  

In [ ]:
signedHash = sym.Transaction.createTransactionHash(signedPayload,Buffer.from(generationHash, 'hex'));
cosignSignedTxs = [
    new sym.CosignatureSignedTransaction(signedHash,bobSignedTxSignature,bobSignedTxSignerPublicKey)
];
recreatedTx = sym.TransactionMapping.createFromPayload(signedPayload);
cosignSignedTxs.forEach((cosignedTx) => {
    signedPayload += cosignedTx.version.toHex() + cosignedTx.signerPublicKey + cosignedTx.signature;
});
size = `00000000${(signedPayload.length / 2).toString(16)}`;
formatedSize = size.substr(size.length - 8, size.length);
littleEndianSize = formatedSize.substr(6, 2) + formatedSize.substr(4, 2) + formatedSize.substr(2, 2) + formatedSize.substr(0, 2);
signedPayload = littleEndianSize + signedPayload.substr(8, signedPayload.length - 8);
signedTx = new sym.SignedTransaction(signedPayload, signedHash, alice.publicKey, recreatedTx.type, recreatedTx.networkType);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加


後半部分の連署を追加する部分がPayload(サイズ値)を直接操作しているので少し難しいかもしれません。
Aliceの秘密鍵で再度署名できる場合はcosignSignedTxsを生成した後、以下のように連署済みトランザクションを生成することも可能です。

```js
resignedTx = recreatedTx.signTransactionGivenSignatures(alice, cosignSignedTxs, generationHash);
await txRepo.announce(resignedTx).toPromise();
```

## 12.4 現場で使えるヒント

### マーケットプレイスレス
ボンデッドトランザクションと異なりハッシュロックのための手数料(10XYM)を気にする必要がありません。    
ペイロードを共有できる場が存在する場合、売り手は考えられるすべての買い手候補に対してペイロードを作成して交渉開始を待つことができます。
（複数のトランザクションが個別に実行されないように、1つしか存在しない領収書NFTをアグリゲートトランザクションに混ぜ込むなどして排他制御をしてください）。
この交渉に専用のマーケットプレイスを構築する必要はありません。
SNSのタイムラインをマーケットプレイスにしたり、必要に応じて任意の時間や空間でワンタイムマーケットプレイスを展開することができます。  

ただ、オフラインで署名を交換するため、なりすましのハッシュ署名要求には気を付けましょう。  
（必ず検証可能なペイロードからハッシュを生成して署名するようにしてください）  

# <span style="color:red">チャットでオフライン署名</span>　　
<span style="color:red">ローカルではなく、実際にチャットを使って講師のaliceから署名付きペイロードを受け取り、ローカルのbobで連署及びアナウンスしてみる</span>  
### <span style="color:red">これが実現できればユーザがXYMを持たずしてブロックチェーンを利用するサービスを構築できる</span>  

<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/230841022-a5f72b39-bc14-453e-bc8d-b162ee9ebc38.png">
<img width="800" alt="スクリーンショット 2023-04-21 4 52 14" src="https://user-images.githubusercontent.com/47712051/230847070-3e0bc4fd-0cd6-4c3c-b24e-7b3e60c36c15.png">


## <span style="color:red">トランザクション作成</span>  
<span style="color:red">bobのアドレスと公開鍵を確認し、Discordのチャットで講師に伝える（受講者作業）</span>

In [ ]:
console.log(`${bob.address.plain()}`) 
console.log(`${bob.publicKey}`) 

<span style="color:red">講師側でペイロードを作成し、受講者にペイロードと公開鍵をzoomチャットで伝える（講師作業）</span>

In [ ]:
bobAddress = sym.Address.createFromRawAddress("🌟ここに受講者のアドレスを入力")
bobPublicAccount = sym.PublicAccount.createFromPublicKey("🌟ここに受講者の公開鍵を入力",networkType);
  
innerTx1 = sym.TransferTransaction.create(
    undefined,
    bobAddress, 
    [],
    sym.PlainMessage.create("講師=>受講者向けメッセージ"),
    networkType
);
innerTx2 = sym.TransferTransaction.create(
    undefined,
    alice.address, 
    [],
    sym.PlainMessage.create("受講者=>講師向けメッセージ"),
    networkType
);
aggregateTx = sym.AggregateTransaction.createComplete(
    sym.Deadline.create(epochAdjustment),
    [
      innerTx1.toAggregate(alice.publicAccount),
      innerTx2.toAggregate(bobPublicAccount)
    ],
    networkType,
    [],
).setMaxFeeForAggregate(100, 1);
signedTx =  alice.sign(aggregateTx,generationHash);
signedHash = signedTx.hash;
signedPayload = signedTx.payload;

console.log(alice.publicAccount);
console.log(signedPayload);

## <span style="color:red">Bobによる連署 (受講者作業)</span>  

In [ ]:
signedPayload = "🌟ここに講師よりDiscordで受け取ったペイロードを記載"
tx = sym.TransactionMapping.createFromPayload(signedPayload);
console.log(tx);

In [ ]:
res = tx.signer.verifySignature(
    tx.getSigningBytes([...Buffer.from(signedPayload,'hex')],[...Buffer.from(generationHash,'hex')]),
    tx.signature
);
console.log(res);

In [ ]:
bobSignedTx = sym.CosignatureTransaction.signTransactionPayload(bob, signedPayload, generationHash);
bobSignedTxSignature = bobSignedTx.signature;
bobSignedTxSignerPublicKey = bobSignedTx.signerPublicKey;

In [ ]:
alicePublicAccount = sym.PublicAccount.createFromPublicKey("🌟ここにDiscordで講師に教えてもらった公開鍵を入力",networkType);

signedHash = sym.Transaction.createTransactionHash(signedPayload,Buffer.from(generationHash, 'hex'));
cosignSignedTxs = [
    new sym.CosignatureSignedTransaction(signedHash,bobSignedTxSignature,bobSignedTxSignerPublicKey)
];
recreatedTx = sym.TransactionMapping.createFromPayload(signedPayload);
cosignSignedTxs.forEach((cosignedTx) => {
    signedPayload += cosignedTx.version.toHex() + cosignedTx.signerPublicKey + cosignedTx.signature;
});
size = `00000000${(signedPayload.length / 2).toString(16)}`;
formatedSize = size.substr(size.length - 8, size.length);
littleEndianSize = formatedSize.substr(6, 2) + formatedSize.substr(4, 2) + formatedSize.substr(2, 2) + formatedSize.substr(0, 2);
signedPayload = littleEndianSize + signedPayload.substr(8, signedPayload.length - 8);
signedTx = new sym.SignedTransaction(signedPayload, signedHash, alicePublicAccount, recreatedTx.type, recreatedTx.networkType);
await txRepo.announce(signedTx).toPromise();

In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加